In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

In [ ]:
data = pd.read_csv("df_imputed_nodrop.csv")
data.drop(columns=['Unnamed: 0'], inplace=True)
data

In [ ]:
#Estimated mean and median ICU length of stay
# Convert 'icu_intime' and 'icu_outtime' columns to datetime
data['icu_intime'] = pd.to_datetime(data['icu_intime'])
data['icu_outtime'] = pd.to_datetime(data['icu_outtime'])

# Calculate ICU stay length in days for each patient
data['icu_stay_length'] = (data['icu_outtime'] - data['icu_intime']).dt.total_seconds() / (24 * 3600)

# Display statistics on ICU stay lengths
icu_stay_statistics = data['icu_stay_length'].describe()
icu_stay_statistics

Death Patient Dataset

In [ ]:
# Remove rows with negative ICU stay lengths
data_cleaned = data[data['icu_stay_length'] >= 0]

# Filter out the patients who died in ICU
data_icu_deaths = data_cleaned[data_cleaned['icu_death'] == 1]

# Checking the cleaned and filtered data
data_icu_deaths.describe(include='all')

In [ ]:
#Create a time window with a length of 3

# Ensure 'charttime' is in datetime format
data_icu_deaths['charttime'] = pd.to_datetime(data_icu_deaths['charttime'])

# Sort by patient ID and charttime to ensure data is in chronological order
data_icu_deaths_sorted = data_icu_deaths.sort_values(by=['id', 'charttime'])

# Define a function to create rolling windows of 3 days
def create_rolling_windows_with_avg(df, window_size=3):
    windows = []
    avg_los_icu = []
    for patient_id, group in df.groupby('id'):
        for start_index in range(len(group) - window_size + 1):
            window = group.iloc[start_index:start_index + window_size]
            if (window['charttime'].iloc[-1] - window['charttime'].iloc[0]).days < window_size:
                windows.append(window)
                avg_los_icu.append(window['los_icu'].mean())  # 计算窗口内 los_icu 的平均值
    return windows, avg_los_icu

# Apply this function to create time windows and averages
time_windows, avg_los_icu = create_rolling_windows_with_avg(data_icu_deaths_sorted)

# Example of how many windows we have and a peek at the first window data
len(time_windows), time_windows[0].head()

In [ ]:
# Remove non-numeric columns for correlation computation
numeric_data = data_icu_deaths.select_dtypes(include=[np.number])

# Calculate correlation matrix
correlation_matrix = numeric_data.corr()

# Find the top 30 features most correlated with los_icu, excluding 'id' and 'los_icu' itself
top_features = correlation_matrix['los_icu'].drop(['id', 'los_icu']).abs().sort_values(ascending=False).head(31)

# Display the top 30 features
top_features = top_features.iloc[1:]
top_features

In [ ]:
selected_features = top_features.index.tolist()

# Re-filtering the data to include only the selected top features, plus 'los_icu', 'charttime', 'id'
filtered_data_windows = [window[selected_features + ['los_icu', 'charttime', 'id']] for window in time_windows]

# Combine all windows into a single DataFrame
combined_data = pd.concat(filtered_data_windows)

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(combined_data[selected_features])
scaled_target = combined_data['los_icu'].values

# Convert the list of averages to a numpy array for subsequent processing
y = np.array(avg_los_icu)

# Make sure the shape of the data and labels are consistent
X = scaled_features.reshape(len(time_windows), len(time_windows[0]), len(selected_features))
y = y.reshape(-1, 1)  # Make sure y is a two-dimensional array that adapts to the model output

# Split the data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Shapes of the training and test datasets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
#Adjust the model architecture to ensure that the output layer is suitable for average prediction
# Build LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Model training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')
history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, callbacks=[early_stopping], batch_size=32)

# Model evaluation
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test)

# Calculate the mean squared error, mean absolute error and coefficient of determination
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r_squared = r2_score(y_test, predictions)

print(f'Test MSE: {mse}')
print(f'Test MAE: {mae}')
print("R-squared (Coefficient of Determination):", r_squared)

In [ ]:
#RandomizedSearchCV provides an alternative to GridSearchCV that instead of trying all parameter combinations, randomly selects from a specified parameter distribution. This can significantly reduce the number of configurations that must be evaluated.from scikeras.wrappers import KerasRegressor
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

def build_model(lstm_units=50, activation='relu', optimizer='adam'):
    model = Sequential([
        LSTM(lstm_units, activation=activation, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mse')
    return model

model = KerasRegressor(model=build_model, epochs=100, batch_size=32, verbose=0)

param_distributions = {
    'model__lstm_units': [20, 50, 100],
    'model__activation': ['relu', 'tanh'],
    'model__optimizer': ['adam', 'rmsprop'],
    'batch_size': [32, 64],
    'epochs': [50, 100]
}

rand_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_iter=10, cv=3, scoring='neg_mean_squared_error', random_state=42)

rand_search.fit(X_train, y_train)

print("Best: %f using %s" % (rand_search.best_score_, rand_search.best_params_))

In [ ]:
#Adjust the model architecture to ensure that the output layer is suitable for average prediction
model = Sequential([
    LSTM(100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')
history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, callbacks=[early_stopping], batch_size=64)

loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r_squared = r2_score(y_test, predictions)

print(f'Test MSE: {mse}')
print(f'Test MAE: {mae}')
print("R-squared (Coefficient of Determination):", r_squared)

Non-death Patient Dataset

In [ ]:
# Remove rows with negative ICU stay lengths
data_cleaned = data[data['icu_stay_length'] >= 0]

# Filter out the patients who died in ICU
data_icu_survived = data_cleaned[data_cleaned['icu_death'] == 0]

# Checking the cleaned and filtered data
data_icu_survived.describe(include='all')

In [ ]:
#Create a time window with a length of 3

# Ensure 'charttime' is in datetime format
data_icu_survived['charttime'] = pd.to_datetime(data_icu_survived['charttime'])

# Sort by patient ID and charttime to ensure data is in chronological order
data_icu_survived_sorted = data_icu_survived.sort_values(by=['id', 'charttime'])

# Define a function to create rolling windows of 3 days
def create_rolling_windows_with_avg(df, window_size=3):
    windows = []
    avg_los_icu = []
    for patient_id, group in df.groupby('id'):
        for start_index in range(len(group) - window_size + 1):
            window = group.iloc[start_index:start_index + window_size]
            if (window['charttime'].iloc[-1] - window['charttime'].iloc[0]).days < window_size:
                windows.append(window)
                avg_los_icu.append(window['los_icu'].mean())  # 计算窗口内 los_icu 的平均值
    return windows, avg_los_icu

# Apply this function to create time windows and averages
time_windows, avg_los_icu = create_rolling_windows_with_avg(data_icu_survived_sorted)

# Example of how many windows we have and a peek at the first window data
len(time_windows), time_windows[0].head()

In [ ]:
# Remove non-numeric columns for correlation computation
numeric_data = data_icu_survived.select_dtypes(include=[np.number])

# Calculate correlation matrix
correlation_matrix = numeric_data.corr()

# Find the top 30 features most correlated with los_icu, excluding 'id' and 'los_icu' itself
top_features = correlation_matrix['los_icu'].drop(['id', 'los_icu']).abs().sort_values(ascending=False).head(31)

# Display the top 30 features
top_features = top_features.iloc[1:]
top_features

In [ ]:
selected_features = top_features.index.tolist()

# Re-filtering the data to include only the selected top features, plus 'los_icu', 'charttime', 'id'
filtered_data_windows = [window[selected_features + ['los_icu', 'charttime', 'id']] for window in time_windows]

# Combine all windows into a single DataFrame
combined_data = pd.concat(filtered_data_windows)

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(combined_data[selected_features])
scaled_target = combined_data['los_icu'].values

# Convert the list of averages to a numpy array for subsequent processing
y = np.array(avg_los_icu)

X = scaled_features.reshape(len(time_windows), len(time_windows[0]), len(selected_features))
y = y.reshape(-1, 1)

# Split the data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Shapes of the training and test datasets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from scikeras.wrappers import KerasRegressor
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

def build_model(lstm_units=50, activation='relu', optimizer='adam'):
    model = Sequential([
        LSTM(lstm_units, activation=activation, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mse')
    return model

model = KerasRegressor(model=build_model, epochs=100, batch_size=32, verbose=0)

param_distributions = {
    'model__lstm_units': [20, 50, 100],
    'model__activation': ['relu', 'tanh'],
    'model__optimizer': ['adam', 'rmsprop'],
    'batch_size': [32, 64],
    'epochs': [50, 100]
}

rand_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_iter=10, cv=3, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

rand_result = rand_search.fit(X_train, y_train)

print("Best: %f using %s" % (rand_result.best_score_, rand_result.best_params_))

In [ ]:
#Adjust the model architecture to ensure that the output layer is suitable for average prediction
# Build LSTM model
model = Sequential([
    LSTM(100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Model training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')
history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, callbacks=[early_stopping], batch_size=32)

# Model evaluation
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions
predictions = model.predict(X_test)

# Calculate the mean squared error, mean absolute error and coefficient of determination
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r_squared = r2_score(y_test, predictions)

print(f'Test MSE: {mse}')
print(f'Test MAE: {mae}')
print("R-squared (Coefficient of Determination):", r_squared)